In [ ]:
#Install all necessery packages
!pip install nltk wikipedia-api
import nltk
import wikipediaapi
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize
from collections import Counter

#Download all necessery data
nltk.download('wordnet')
nltk.download('punkt')

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.7.1-py3-none-any.whl size=14346 sha256=e8ddac7deaf817a80e15799d1f6f7ecf0d6e18d512236cd23072f89e3060aca4
  Stored in directory: /root/.cache/pip/wheels/4c/96/18/b9201cc3e8b47b02b510460210cfd832ccf10c0c4dd0522962
Successfully built wikipedia-api


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Task-1: Lesk disambiguation

In [ ]:
# Natural Language Toolkit: Word Sense Disambiguation Algorithms
#
# Authors: Liling Tan <alvations@gmail.com>,
#          Dmitrijs Milajevs <dimazest@gmail.com>
#
# Copyright (C) 2001-2024 NLTK Project
# URL: <https://www.nltk.org/>
# For license information, see LICENSE.TXT

from nltk.corpus import wordnet




def lesk(context_sentence, ambiguous_word, pos=None, synsets=None, lang="eng"):
    """Return a synset for an ambiguous word in a context.

    :param iter context_sentence: The context sentence where the ambiguous word
         occurs, passed as an iterable of words.
    :param str ambiguous_word: The ambiguous word that requires WSD.
    :param str pos: A specified Part-of-Speech (POS).
    :param iter synsets: Possible synsets of the ambiguous word.
    :param str lang: WordNet language.
    :return: ``lesk_sense`` The Synset() object with the highest signature overlaps.

    This function is an implementation of the original Lesk algorithm (1986) [1].

    Usage example::

        >>> lesk(['I', 'went', 'to', 'the', 'bank', 'to', 'deposit', 'money', '.'], 'bank', 'n')
        Synset('savings_bank.n.02')

    [1] Lesk, Michael. "Automatic sense disambiguation using machine
    readable dictionaries: how to tell a pine cone from an ice cream
    cone." Proceedings of the 5th Annual International Conference on
    Systems Documentation. ACM, 1986.
    https://dl.acm.org/citation.cfm?id=318728
    """

    context = set(context_sentence)
    if synsets is None:
        synsets = wordnet.synsets(ambiguous_word, lang=lang)

    if pos:
        synsets = [ss for ss in synsets if str(ss.pos()) == pos]

    if not synsets:
        return None

    _, sense = max(
        (len(context.intersection(ss.definition().split())), ss) for ss in synsets
    )

    return sense

In [ ]:
sentence = "I was awarded a chair in computer science."
target_word = "chair"
correct_sense = lesk(sentence, target_word)
print(correct_sense)
print(correct_sense.definition())

Synset('chair.v.01')
act or preside as chair, as of an academic department in a university


Task-2: Wikipedia base disambiguation using Lesk algorithm

In [ ]:
#modified lesk implementation
def lesk_similarity(context_sentence, sense_defination):
  context_tokens = set(word_tokenize(context_sentence.lower()))
  sense_def_tokens = set(word_tokenize(sense_defination.lower()))
  overlap = len(sense_def_tokens.intersection(context_tokens))
  return overlap

In [ ]:
#search the target word on wikipedia
agent = "Project_Task_1_2_3/1.0 (mdrabiulhasan7890@gmail.com)"
wiki_wiki = wikipediaapi.Wikipedia(language='en',
                                   extract_format=wikipediaapi.ExtractFormat.WIKI,
                                   user_agent=agent)

def search_targetword_on_wiki(word):
  main_page = wiki_wiki.page(f"{word} (disambiguation)")
  senses = []
  if main_page.exists():
    for link in main_page.links.values():
      sense_page = wiki_wiki.page(link.title)
      if sense_page.exists() and sense_page.summary:
        senses.append((link.title, sense_page.text))

  else:
      main_page = wiki_wiki.page(word)
      if main_page.exists() and main_page.summary:
        senses.append((word, main_page.text))
  return senses


#Use wiki for lexical resourse (sense, text) for disambiguation
def wiki_disambiguation(sentence, target_word):
  senses = search_targetword_on_wiki(target_word)
  max_overlap = 0
  best_sense = None
  for title, text in senses:
    overlap = lesk_similarity(sentence, text)
    if overlap > max_overlap:
      max_overlap = overlap
      best_sense = (title, text)
  return best_sense


In [ ]:
!pip install summa
from summa import summarizer

sentence = "I was awarded a chair in computer science."
target_word = "chair"

wiki_based_best_sense = wiki_disambiguation(sentence, target_word)

text = wiki_based_best_sense[1]
summary = summarizer.summarize(text, ratio=0.01)

print("Best Sense:", wiki_based_best_sense[0])
print("Summary:", summary)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54387 sha256=9c4afa68ffff417565a7580ca81ad05340aeef2ea2c7c6b097abe937bc86abc7
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa
Best Sense: Chair (academic)
Summary: Professor (commonly abbreviated as Prof.) is an academic rank at universities and other post-secondary education and research institutions in most countries.
